## Table of Contents

1. [Environment Setup & Random Seed Configuration](#1.-Environment-Setup-&-Random-Seed-Configuration)
2. [Import Libraries](#2.-Import-Libraries)
3. [Dataset Download & Verification](#3.-Dataset-Download-&-Verification)
4. [Exploratory Data Analysis (EDA)](#4.-Exploratory-Data-Analysis-(EDA))
5. [Data Preprocessing & Subset Creation](#5.-Data-Preprocessing-&-Subset-Creation)
6. [Data Augmentation](#6.-Data-Augmentation)
7. [YOLOv8 Object Detection](#7.-YOLOv8-Object-Detection)
8. [U-Net Semantic Segmentation](#8.-U-Net-Semantic-Segmentation)
9. [Model Comparison & Discussion](#9.-Model-Comparison-&-Discussion)
10. [Conclusions & Future Work](#10.-Conclusions-&-Future-Work)

---
## 1. Environment Setup & Random Seed Configuration

Setting all random seeds for reproducibility across:
- Python's built-in random module
- NumPy
- PyTorch (CPU and CUDA)
- Python hash seed

In [ ]:
# Set environment variable for Python hash seed (must be done before importing libraries)
import os
os.environ['PYTHONHASHSEED'] = '0'

# Import random libraries
import random
import numpy as np

# Set random seeds
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# PyTorch seeds (will be set after importing torch)
print("✓ Python hash seed set to 0")
print(f"✓ Random seed set to {RANDOM_SEED}")
print(f"✓ NumPy seed set to {RANDOM_SEED}")

In [ ]:
# Import PyTorch and set its random seeds
import torch

torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

# Additional PyTorch reproducibility settings
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"✓ PyTorch seed set to {RANDOM_SEED}")
print(f"✓ PyTorch CUDA seed set to {RANDOM_SEED}")
print("✓ CUDNN deterministic mode enabled")
print("✓ CUDNN benchmark disabled for reproducibility")

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✓ Using device: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA Version: {torch.version.cuda}")

---
## 2. Import Libraries

Importing all required libraries for:
- Data manipulation and analysis
- Image processing
- Deep learning (PyTorch, YOLOv8)
- Visualization
- COCO dataset handling

In [ ]:
# Data manipulation and analysis
import pandas as pd
from collections import Counter, defaultdict
import json
from pathlib import Path
import shutil
from tqdm import tqdm

# Image processing
import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Deep Learning - PyTorch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

# YOLOv8
from ultralytics import YOLO

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

# COCO tools
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

# Metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully!")
print(f"\nLibrary Versions:")
print(f"  PyTorch: {torch.__version__}")
print(f"  Torchvision: {torchvision.__version__}")
print(f"  NumPy: {np.__version__}")
print(f"  Pandas: {pd.__version__}")
print(f"  OpenCV: {cv2.__version__}")
print(f"  Albumentations: {A.__version__}")

---
## 3. Dataset Download & Verification

### 3.1 Download TACO Dataset

The TACO dataset can be downloaded using:
1. **Kaggle API** (recommended - automated)
2. **Manual download** from Kaggle website

In [ ]:
# Define project directories
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
TACO_DIR = DATA_DIR / 'TACO'
RUNS_DIR = PROJECT_ROOT / 'runs'
WEIGHTS_DIR = PROJECT_ROOT / 'weights'

# Create directories
DATA_DIR.mkdir(exist_ok=True)
RUNS_DIR.mkdir(exist_ok=True)
WEIGHTS_DIR.mkdir(exist_ok=True)

print("Project directory structure:")
print(f"  Project Root: {PROJECT_ROOT}")
print(f"  Data Directory: {DATA_DIR}")
print(f"  TACO Directory: {TACO_DIR}")
print(f"  Runs Directory: {RUNS_DIR}")
print(f"  Weights Directory: {WEIGHTS_DIR}")

In [ ]:
# Check if dataset already exists
if TACO_DIR.exists() and len(list(TACO_DIR.glob('*'))) > 0:
    print("✓ TACO dataset already exists!")
    print(f"  Location: {TACO_DIR}")
else:
    print("Dataset not found. Please download using one of the following methods:\n")
    
    print("Method 1: Kaggle API (Recommended)")
    print("-" * 50)
    print("1. Install Kaggle API: pip install kaggle")
    print("2. Setup Kaggle credentials (kaggle.json)")
    print("3. Run the following commands:\n")
    print("   kaggle datasets download -d kneroma/tacotrashdataset")
    print(f"   unzip tacotrashdataset.zip -d {DATA_DIR}")
    print("\nMethod 2: Manual Download")
    print("-" * 50)
    print("1. Visit: https://www.kaggle.com/datasets/kneroma/tacotrashdataset")
    print("2. Download the dataset")
    print(f"3. Extract to: {DATA_DIR}")
    print("\nNote: Uncomment and run the cell below to download via Kaggle API")

In [ ]:
# Uncomment to download using Kaggle API
# !pip install kaggle
# !kaggle datasets download -d kneroma/tacotrashdataset
# import zipfile
# with zipfile.ZipFile('tacotrashdataset.zip', 'r') as zip_ref:
#     zip_ref.extractall(DATA_DIR)
# print("✓ Dataset downloaded and extracted successfully!")

### 3.2 Verify Dataset Structure and COCO Annotations

In [ ]:
# Verify dataset structure
if TACO_DIR.exists():
    print("Dataset Structure:")
    print("-" * 50)
    
    # List all subdirectories and files
    for item in sorted(TACO_DIR.glob('*')):
        if item.is_dir():
            file_count = len(list(item.glob('*')))
            print(f"📁 {item.name}/ ({file_count} items)")
        else:
            file_size = item.stat().st_size / (1024 * 1024)  # Convert to MB
            print(f"📄 {item.name} ({file_size:.2f} MB)")
    
    # Look for annotation files
    annotation_files = list(TACO_DIR.rglob('*.json'))
    print(f"\nFound {len(annotation_files)} annotation file(s):")
    for ann_file in annotation_files:
        print(f"  - {ann_file.relative_to(TACO_DIR)}")
else:
    print("⚠️ Dataset not found. Please download the dataset first.")

In [ ]:
# Load and verify COCO annotations
# Note: Update the annotation file path based on actual dataset structure

# Common TACO annotation file paths
possible_ann_paths = [
    TACO_DIR / 'annotations.json',
    TACO_DIR / 'annotations' / 'instances_default.json',
    TACO_DIR / 'TACO' / 'annotations.json',
]

ANNOTATION_FILE = None
for path in possible_ann_paths:
    if path.exists():
        ANNOTATION_FILE = path
        break

if ANNOTATION_FILE:
    print(f"✓ Found annotation file: {ANNOTATION_FILE.name}")
    
    # Load COCO annotations
    with open(ANNOTATION_FILE, 'r') as f:
        coco_data = json.load(f)
    
    print("\nCOCO Format Verification:")
    print("-" * 50)
    print(f"  Images: {len(coco_data.get('images', []))}")
    print(f"  Annotations: {len(coco_data.get('annotations', []))}")
    print(f"  Categories: {len(coco_data.get('categories', []))}")
    
    # Display first few categories
    print("\nSample Categories:")
    for cat in coco_data.get('categories', [])[:10]:
        print(f"  ID {cat['id']}: {cat['name']}")
    
    if len(coco_data.get('categories', [])) > 10:
        print(f"  ... and {len(coco_data.get('categories', [])) - 10} more")
    
    print("\n✓ COCO format verified successfully!")
else:
    print("⚠️ Annotation file not found. Please verify dataset structure.")
    print("   Expected locations:")
    for path in possible_ann_paths:
        print(f"   - {path}")

### 3.3 Find Image Directory

In [ ]:
# Locate image directory
possible_img_dirs = [
    TACO_DIR / 'images',
    TACO_DIR / 'data',
    TACO_DIR / 'TACO' / 'images',
    TACO_DIR,
]

IMAGE_DIR = None
for img_dir in possible_img_dirs:
    if img_dir.exists():
        # Check if directory contains image files
        img_files = list(img_dir.glob('*.jpg')) + list(img_dir.glob('*.png'))
        if len(img_files) > 0:
            IMAGE_DIR = img_dir
            print(f"✓ Found image directory: {IMAGE_DIR.name}")
            print(f"  Total images: {len(img_files)}")
            break

if not IMAGE_DIR:
    print("⚠️ Image directory not found. Please verify dataset structure.")

---
## 4. Exploratory Data Analysis (EDA)

### 4.1 Load COCO Annotations

Loading the TACO dataset annotations and extracting key statistics.

In [ ]:
# Load COCO annotations
if ANNOTATION_FILE and ANNOTATION_FILE.exists():
    print(f"Loading annotations from: {ANNOTATION_FILE.name}")
    print("=" * 70)
    
    # Initialize COCO API
    coco = COCO(str(ANNOTATION_FILE))
    
    # Get all category IDs and image IDs
    cat_ids = coco.getCatIds()
    img_ids = coco.getImgIds()
    ann_ids = coco.getAnnIds()
    
    # Load categories and images
    categories = coco.loadCats(cat_ids)
    images = coco.loadImgs(img_ids)
    annotations = coco.loadAnns(ann_ids)
    
    print(f"\n✓ Successfully loaded TACO dataset!")
    print(f"\nDataset Statistics:")
    print(f"  Total Images: {len(images)}")
    print(f"  Total Annotations: {len(annotations)}")
    print(f"  Total Categories: {len(categories)}")
    print(f"  Average annotations per image: {len(annotations)/len(images):.2f}")
    
    # Create category name mapping
    cat_id_to_name = {cat['id']: cat['name'] for cat in categories}
    cat_name_to_id = {cat['name']: cat['id'] for cat in categories}
    
    print(f"\n✓ Category mapping created")
    print(f"  Sample categories: {list(cat_id_to_name.items())[:5]}")
    
else:
    print("⚠️ Annotation file not found. Please download the TACO dataset first.")
    print("Stopping execution - dataset is required for EDA.")

### 4.2 Compute Class Frequencies and Statistics

In [ ]:
# Compute class frequencies
class_counts = Counter()
image_object_counts = defaultdict(int)

for ann in annotations:
    class_counts[ann['category_id']] += 1
    image_object_counts[ann['image_id']] += 1

# Convert to sorted list
class_freq_list = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)

# Create DataFrame for better visualization
class_freq_df = pd.DataFrame([
    {
        'Category ID': cat_id,
        'Category Name': cat_id_to_name.get(cat_id, 'Unknown'),
        'Count': count,
        'Percentage': (count / len(annotations)) * 100
    }
    for cat_id, count in class_freq_list
])

print("Class Frequency Distribution (Top 20):")
print("=" * 70)
print(class_freq_df.head(20).to_string(index=False))

print(f"\n\nTop 5 Most Frequent Classes (IDs 0-4 as required):")
print("=" * 70)
top_5_classes = class_freq_list[:5]
for rank, (cat_id, count) in enumerate(top_5_classes, 1):
    cat_name = cat_id_to_name.get(cat_id, 'Unknown')
    percentage = (count / len(annotations)) * 100
    print(f"{rank}. ID {cat_id}: {cat_name:30s} - {count:5d} annotations ({percentage:5.2f}%)")

# Store top 5 class IDs for later use
TOP_5_CLASS_IDS = [cat_id for cat_id, _ in top_5_classes]
print(f"\n✓ Top 5 class IDs stored: {TOP_5_CLASS_IDS}")

In [ ]:
# Analyze objects per image statistics
objects_per_img = list(image_object_counts.values())

print("Objects per Image Statistics:")
print("=" * 70)
print(f"  Mean objects per image: {np.mean(objects_per_img):.2f}")
print(f"  Median objects per image: {np.median(objects_per_img):.2f}")
print(f"  Std deviation: {np.std(objects_per_img):.2f}")
print(f"  Min objects in an image: {min(objects_per_img)}")
print(f"  Max objects in an image: {max(objects_per_img)}")
print(f"  Total images: {len(image_object_counts)}")

# Distribution breakdown
dist_breakdown = Counter(objects_per_img)
print(f"\nDistribution breakdown:")
for num_objs in sorted(dist_breakdown.keys())[:10]:
    count = dist_breakdown[num_objs]
    print(f"  {num_objs} object(s): {count} images ({count/len(images)*100:.1f}%)")
if len(dist_breakdown) > 10:
    print(f"  ... and {len(dist_breakdown) - 10} more categories")

### 4.3 Visualization: Class Distribution Bar Chart

In [ ]:
# Create bar chart for top 20 classes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))

# Plot 1: Top 20 classes
top_20_df = class_freq_df.head(20)
bars1 = ax1.barh(range(len(top_20_df)), top_20_df['Count'], color='steelblue')

# Highlight top 5 classes
for i in range(min(5, len(top_20_df))):
    bars1[i].set_color('coral')
    bars1[i].set_edgecolor('darkred')
    bars1[i].set_linewidth(2)

ax1.set_yticks(range(len(top_20_df)))
ax1.set_yticklabels([f"ID {row['Category ID']}: {row['Category Name'][:25]}" 
                      for _, row in top_20_df.iterrows()])
ax1.set_xlabel('Number of Annotations', fontsize=12, fontweight='bold')
ax1.set_title('Top 20 Classes by Frequency\n(Top 5 highlighted in coral)', 
              fontsize=14, fontweight='bold')
ax1.invert_yaxis()
ax1.grid(axis='x', alpha=0.3)

# Add count labels
for i, (_, row) in enumerate(top_20_df.iterrows()):
    ax1.text(row['Count'], i, f" {row['Count']}", 
             va='center', fontsize=9, fontweight='bold')

# Plot 2: Top 5 classes only (required subset)
top_5_df = class_freq_df.head(5)
bars2 = ax2.bar(range(len(top_5_df)), top_5_df['Count'], 
                color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'],
                edgecolor='black', linewidth=2)

ax2.set_xticks(range(len(top_5_df)))
ax2.set_xticklabels([f"ID {row['Category ID']}\n{row['Category Name'][:20]}" 
                      for _, row in top_5_df.iterrows()], rotation=45, ha='right')
ax2.set_ylabel('Number of Annotations', fontsize=12, fontweight='bold')
ax2.set_title('Top 5 Most Frequent Classes (Subset for Training)', 
              fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Add count and percentage labels
for i, (_, row) in enumerate(top_5_df.iterrows()):
    ax2.text(i, row['Count'], f"{row['Count']}\n({row['Percentage']:.1f}%)", 
             ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(RUNS_DIR / 'class_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Class distribution chart saved to: {RUNS_DIR / 'class_distribution.png'}")

### 4.4 Visualization: Objects per Image Histogram

In [ ]:
# Create histogram of objects per image
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Histogram with all data
ax1.hist(objects_per_img, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
ax1.axvline(np.mean(objects_per_img), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {np.mean(objects_per_img):.2f}')
ax1.axvline(np.median(objects_per_img), color='green', linestyle='--', linewidth=2, 
            label=f'Median: {np.median(objects_per_img):.2f}')
ax1.set_xlabel('Number of Objects per Image', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frequency (Number of Images)', fontsize=12, fontweight='bold')
ax1.set_title('Distribution of Objects per Image (All Data)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# Box plot for better visualization of outliers
ax2.boxplot(objects_per_img, vert=True, patch_artist=True,
            boxprops=dict(facecolor='lightcoral', alpha=0.7),
            medianprops=dict(color='darkred', linewidth=2),
            whiskerprops=dict(linewidth=1.5),
            capprops=dict(linewidth=1.5))
ax2.set_ylabel('Number of Objects per Image', fontsize=12, fontweight='bold')
ax2.set_title('Box Plot: Objects per Image\n(Shows outliers and quartiles)', 
              fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Add statistics text
stats_text = f"Min: {min(objects_per_img)}\n"
stats_text += f"Q1: {np.percentile(objects_per_img, 25):.1f}\n"
stats_text += f"Median: {np.median(objects_per_img):.1f}\n"
stats_text += f"Q3: {np.percentile(objects_per_img, 75):.1f}\n"
stats_text += f"Max: {max(objects_per_img)}"
ax2.text(1.15, np.median(objects_per_img), stats_text, 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
         fontsize=10, verticalalignment='center')

plt.tight_layout()
plt.savefig(RUNS_DIR / 'objects_per_image_histogram.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Objects per image histogram saved to: {RUNS_DIR / 'objects_per_image_histogram.png'}")

### 4.5 Visualization: Sample Images with Bounding Boxes and Masks

In [ ]:
# Helper function to visualize image with annotations
def visualize_annotations(img_id, coco_obj, img_dir, ax_bbox=None, ax_mask=None):
    """
    Visualize bounding boxes and segmentation masks for a given image.
    
    Args:
        img_id: COCO image ID
        coco_obj: COCO API object
        img_dir: Path to image directory
        ax_bbox: Matplotlib axis for bounding box visualization
        ax_mask: Matplotlib axis for mask visualization
    """
    # Load image info
    img_info = coco_obj.loadImgs(img_id)[0]
    img_path = img_dir / img_info['file_name']
    
    # Check if image exists
    if not img_path.exists():
        print(f"Warning: Image not found: {img_path}")
        return False
    
    # Load image
    img = cv2.imread(str(img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Get annotations for this image
    ann_ids = coco_obj.getAnnIds(imgIds=img_id)
    anns = coco_obj.loadAnns(ann_ids)
    
    # Visualization with bounding boxes
    if ax_bbox is not None:
        ax_bbox.imshow(img)
        ax_bbox.axis('off')
        
        for ann in anns:
            # Draw bounding box
            bbox = ann['bbox']  # [x, y, width, height]
            rect = patches.Rectangle(
                (bbox[0], bbox[1]), bbox[2], bbox[3],
                linewidth=2, edgecolor='lime', facecolor='none'
            )
            ax_bbox.add_patch(rect)
            
            # Add label
            cat_name = cat_id_to_name.get(ann['category_id'], 'Unknown')
            ax_bbox.text(
                bbox[0], bbox[1] - 5,
                f"{cat_name[:15]}", 
                color='white', fontsize=8, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='lime', alpha=0.7)
            )
        
        ax_bbox.set_title(f"Bounding Boxes\n{img_info['file_name']}\n{len(anns)} objects", 
                         fontsize=10, fontweight='bold')
    
    # Visualization with segmentation masks
    if ax_mask is not None:
        ax_mask.imshow(img)
        ax_mask.axis('off')
        
        # Create colored mask overlay
        mask_img = np.zeros_like(img, dtype=np.float32)
        
        for idx, ann in enumerate(anns):
            if 'segmentation' in ann and ann['segmentation']:
                # Generate random color for this annotation
                color = np.random.rand(3)
                
                # Handle different segmentation formats
                if isinstance(ann['segmentation'], list):
                    # Polygon format
                    for seg in ann['segmentation']:
                        poly = np.array(seg).reshape(-1, 2).astype(np.int32)
                        cv2.fillPoly(mask_img, [poly], color)
                elif isinstance(ann['segmentation'], dict):
                    # RLE format
                    rle = ann['segmentation']
                    mask = coco_mask.decode(rle)
                    mask_img[mask > 0] = color
        
        # Overlay mask on image
        alpha = 0.5
        overlaid = (img * (1 - alpha) + mask_img * 255 * alpha).astype(np.uint8)
        ax_mask.imshow(overlaid)
        
        ax_mask.set_title(f"Segmentation Masks\n{img_info['file_name']}\n{len(anns)} objects", 
                         fontsize=10, fontweight='bold')
    
    return True

print("✓ Visualization helper function defined")

In [ ]:
# Select and visualize 12 example images (2 rows x 6 images)
# Try to get images with varying number of objects
sample_img_ids = []
for target_count in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
    # Find images with approximately this many objects
    candidates = [img_id for img_id, count in image_object_counts.items() 
                 if abs(count - target_count) <= 2]
    if candidates:
        sample_img_ids.append(random.choice(candidates))
    
    if len(sample_img_ids) >= 12:
        break

# If we don't have enough, just take random ones
if len(sample_img_ids) < 12:
    remaining = 12 - len(sample_img_ids)
    additional = random.sample(list(image_object_counts.keys()), remaining)
    sample_img_ids.extend(additional)

sample_img_ids = sample_img_ids[:12]

print(f"Selected {len(sample_img_ids)} sample images for visualization")
print(f"Sample image IDs: {sample_img_ids[:6]}...")

In [ ]:
# Visualize with bounding boxes
fig, axes = plt.subplots(2, 6, figsize=(24, 8))
axes = axes.flatten()

print("Visualizing bounding boxes...")
for idx, img_id in enumerate(sample_img_ids):
    visualize_annotations(img_id, coco, IMAGE_DIR, ax_bbox=axes[idx])

plt.suptitle('Sample Images with Bounding Boxes', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(RUNS_DIR / 'sample_images_bboxes.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Bounding box visualization saved to: {RUNS_DIR / 'sample_images_bboxes.png'}")

In [ ]:
# Visualize with segmentation masks
fig, axes = plt.subplots(2, 6, figsize=(24, 8))
axes = axes.flatten()

print("Visualizing segmentation masks...")
for idx, img_id in enumerate(sample_img_ids):
    visualize_annotations(img_id, coco, IMAGE_DIR, ax_mask=axes[idx])

plt.suptitle('Sample Images with Segmentation Masks', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig(RUNS_DIR / 'sample_images_masks.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Segmentation mask visualization saved to: {RUNS_DIR / 'sample_images_masks.png'}")

---
## 5. Data Preprocessing & Subset Creation

### 5.1 Why Use Only 5 Classes?

**Justification for selecting top 5 classes:**

1. **Less Label Noise**: Focusing on the most frequent classes reduces annotation inconsistencies and mislabeling that are more common in rare classes.

2. **More Samples Per Class**: Top 5 classes have significantly more training examples, enabling better model learning and generalization.

3. **Simpler Decision Boundary**: Fewer classes reduce inter-class confusion and make it easier for the model to learn discriminative features.

4. **Balanced Training**: The top 5 classes together represent a substantial portion of the dataset, providing sufficient diversity while maintaining focus.

5. **Computational Efficiency**: Training on fewer classes speeds up experimentation and allows for more thorough hyperparameter tuning.

6. **Better Evaluation Metrics**: With more samples per class, evaluation metrics (Precision, Recall, mAP, IoU) are more reliable and statistically significant.

In [ ]:
### 5.2 Filter Dataset to Top 5 Classes

In [ ]:
# Filter annotations to keep only top 5 classes
filtered_annotations = [ann for ann in annotations if ann['category_id'] in TOP_5_CLASS_IDS]

# Get image IDs that have at least one annotation from top 5 classes
filtered_img_ids = set(ann['image_id'] for ann in filtered_annotations)

# Filter images
filtered_images = [img for img in images if img['id'] in filtered_img_ids]

# Filter categories to top 5
filtered_categories = [cat for cat in categories if cat['id'] in TOP_5_CLASS_IDS]

print("Subset Filtering Results:")
print("=" * 70)
print(f"Original dataset:")
print(f"  Images: {len(images)}")
print(f"  Annotations: {len(annotations)}")
print(f"  Categories: {len(categories)}")
print(f"\nFiltered subset (Top 5 classes):")
print(f"  Images: {len(filtered_images)} ({len(filtered_images)/len(images)*100:.1f}%)")
print(f"  Annotations: {len(filtered_annotations)} ({len(filtered_annotations)/len(annotations)*100:.1f}%)")
print(f"  Categories: {len(filtered_categories)}")
print(f"\nTop 5 classes retained:")
for cat_id in TOP_5_CLASS_IDS:
    cat_name = cat_id_to_name.get(cat_id, 'Unknown')
    count = sum(1 for ann in filtered_annotations if ann['category_id'] == cat_id)
    print(f"  ID {cat_id}: {cat_name:30s} - {count:5d} annotations")

# Compute statistics for filtered dataset
filtered_img_obj_counts = Counter()
for ann in filtered_annotations:
    filtered_img_obj_counts[ann['image_id']] += 1

print(f"\nFiltered dataset statistics:")
print(f"  Avg objects per image: {len(filtered_annotations)/len(filtered_images):.2f}")
print(f"  Min objects per image: {min(filtered_img_obj_counts.values())}")
print(f"  Max objects per image: {max(filtered_img_obj_counts.values())}")

### 5.3 Create TACO Subset Directory and Files

In [ ]:
# Create subset directory structure
SUBSET_DIR = DATA_DIR / 'taco_subset'
SUBSET_IMAGES_DIR = SUBSET_DIR / 'images'
SUBSET_ANN_FILE = SUBSET_DIR / 'annotations.json'

# Create directories
SUBSET_DIR.mkdir(exist_ok=True)
SUBSET_IMAGES_DIR.mkdir(exist_ok=True)

print(f"Created subset directory: {SUBSET_DIR}")
print(f"Images will be stored in: {SUBSET_IMAGES_DIR}")
print(f"Annotations will be saved to: {SUBSET_ANN_FILE}")

In [ ]:
# Copy filtered images to subset directory
print("Copying images to subset directory...")
print("=" * 70)

copied_count = 0
failed_count = 0
copied_files = []

for img_info in tqdm(filtered_images, desc="Copying images"):
    src_path = IMAGE_DIR / img_info['file_name']
    dst_path = SUBSET_IMAGES_DIR / img_info['file_name']
    
    if src_path.exists():
        shutil.copy2(src_path, dst_path)
        copied_count += 1
        copied_files.append(img_info['file_name'])
    else:
        print(f"Warning: Source image not found: {src_path}")
        failed_count += 1

print(f"\n✓ Image copying complete!")
print(f"  Successfully copied: {copied_count} images")
if failed_count > 0:
    print(f"  Failed to copy: {failed_count} images")

In [ ]:
# Create filtered COCO annotation JSON
# Re-map annotation and image IDs to be sequential starting from 1
new_img_id_map = {old_id: new_id for new_id, old_id in enumerate(sorted(filtered_img_ids), 1)}
new_ann_id_map = {}

# Create new annotations with remapped IDs
new_annotations = []
for new_ann_id, ann in enumerate(filtered_annotations, 1):
    new_ann = ann.copy()
    new_ann['id'] = new_ann_id
    new_ann['image_id'] = new_img_id_map[ann['image_id']]
    new_annotations.append(new_ann)
    new_ann_id_map[ann['id']] = new_ann_id

# Create new images with remapped IDs
new_images = []
for img in filtered_images:
    new_img = img.copy()
    new_img['id'] = new_img_id_map[img['id']]
    new_images.append(new_img)

# Sort by ID for consistency
new_images.sort(key=lambda x: x['id'])
new_annotations.sort(key=lambda x: x['id'])

# Create new categories (keep original IDs for top 5)
new_categories = filtered_categories

# Build COCO JSON structure
subset_coco_data = {
    'info': {
        'description': 'TACO Subset - Top 5 Most Frequent Classes',
        'version': '1.0',
        'year': 2025,
        'contributor': 'Minahil Ali (22i-0849), Ayaan Khan (22i-0832)',
        'date_created': '2025-11-16'
    },
    'licenses': [],
    'images': new_images,
    'annotations': new_annotations,
    'categories': new_categories
}

# Save to JSON file
with open(SUBSET_ANN_FILE, 'w') as f:
    json.dump(subset_coco_data, f, indent=2)

print(f"✓ Subset annotations saved to: {SUBSET_ANN_FILE}")
print(f"\nSubset COCO JSON contains:")
print(f"  Images: {len(new_images)}")
print(f"  Annotations: {len(new_annotations)}")
print(f"  Categories: {len(new_categories)}")

In [ ]:
# Create subset manifest CSV for easy reference
manifest_data = []

for img in new_images:
    img_anns = [ann for ann in new_annotations if ann['image_id'] == img['id']]
    
    manifest_data.append({
        'image_id': img['id'],
        'file_name': img['file_name'],
        'width': img.get('width', 'N/A'),
        'height': img.get('height', 'N/A'),
        'num_objects': len(img_anns),
        'category_ids': ','.join(str(ann['category_id']) for ann in img_anns),
        'category_names': ','.join(cat_id_to_name.get(ann['category_id'], 'Unknown') 
                                   for ann in img_anns)
    })

manifest_df = pd.DataFrame(manifest_data)
manifest_csv_path = SUBSET_DIR / 'subset_manifest.csv'
manifest_df.to_csv(manifest_csv_path, index=False)

print(f"✓ Subset manifest CSV saved to: {manifest_csv_path}")
print(f"\nManifest preview:")
print(manifest_df.head(10).to_string(index=False))

print(f"\n\n{'='*70}")
print("SUBSET CREATION COMPLETE!")
print(f"{'='*70}")
print(f"Location: {SUBSET_DIR}")
print(f"Files:")
print(f"  - annotations.json: {SUBSET_ANN_FILE}")
print(f"  - subset_manifest.csv: {manifest_csv_path}")
print(f"  - images/: {SUBSET_IMAGES_DIR} ({copied_count} images)")
print(f"{'='*70}")

---
## 6. Data Augmentation

Coming next: Implement augmentation strategies

In [ ]:
# Placeholder for augmentation section
print("Augmentation section will be implemented in the next milestone")

---
## 7. YOLOv8 Object Detection

Coming next: Train and evaluate YOLOv8 model

In [ ]:
# Placeholder for YOLOv8 section
print("YOLOv8 section will be implemented in the next milestone")

---
## 8. U-Net Semantic Segmentation

Coming next: Build and train U-Net model

In [ ]:
# Placeholder for U-Net section
print("U-Net section will be implemented in the next milestone")

---
## 9. Model Comparison & Discussion

Coming next: Compare YOLO vs U-Net performance

In [ ]:
# Placeholder for comparison section
print("Comparison section will be implemented in the next milestone")

---
## 10. Conclusions & Future Work

Coming next: Final conclusions and recommendations

In [ ]:
# Placeholder for conclusions section
print("Conclusions section will be implemented in the next milestone")

---
## End of Notebook

**Project:** YOLOv8 + U-Net Waste Detection and Segmentation  
**Course:** Deep Learning for Perception (CS4045)  
**Authors:** Minahil Ali (22i-0849), Ayaan Khan (22i-0832)  